# A generalised 1D model

There are currently 2 types on 1D models in gammapy
- `TemporalModel`
- `SpectralModel`

with significant code duplication. These have pre-defined parameter units and final evaluation units. 

It can be often useful to fit models on phase/radial distance etc, where the paramterisation remains the same and only the units differ.

This notebook aims to provide an example of general 1 models with a unit less paramterisation. In this example, the `SkyModel` takes a dictionary of models specifying the axis on which the model operates

In [1]:

import numpy as np

import numpy as np
from astropy import units as u
from astropy.time import Time
from gammapy.maps import *
from gammapy.modeling import Parameter, Parameters
from gammapy.utils.scripts import make_path
from gammapy.utils.time import time_ref_from_dict, time_ref_to_dict
from gammapy.modeling.models.core import ModelBase, _build_parameters_from_dict
from gammapy.modeling.models.spectral import integrate_spectrum, PowerLawSpectralModel
from gammapy.modeling.models import PowerLawTemporalModel, SkyModel

In this example, we work with a simple power law model

$\phi(x) = \phi_0 \cdot \left( \frac{x-x_{ref}}{x_0} \right)^{-\alpha}$



In [2]:
def integrate_linear(func, range_min, range_max, ndecade=100, oversampling_factor=100, **kwargs):
        if isinstance(range_min, Time):
            range_min=range_min.mjd * u.d
            range_max=range_max.mjd * u.d
        x, steps = np.linspace(
            range_min, range_max, oversampling_factor, retstep=True, axis=-1
        )
        values = func(x)
        integral = np.sum(values, axis=-1) * steps
        return integral/np.sum(range_max - range_min) 

class PowerLawModel(ModelBase):

    
    alpha = Parameter("alpha", 2, frozen=True)
    x0 = Parameter("x0", 1, frozen=True)
    x_ref = Parameter("x_ref", 1, frozen=True)
    phi0 = Parameter("phi0", 1, frozen=True, is_norm=True, scale_method="scale10",
                    interp="log")
    
    def __init__(self, input_units="", norm_units=""):
        super().__init__()
        par_input_units = ["x0", "x_ref"]
        self.axis_units = input_units
        for p in par_input_units:
            par = getattr(self, p)
            par.unit = input_units
        self.phi0.unit = norm_units

    def __call__(self, x):
        kwargs = {par.name: par.quantity for par in self.parameters}
        if isinstance(x, Time):
            x=Time(x, scale=x.scale).mjd * u.d
        x = x.to(self.axis_units)
        return self.evaluate(x, **kwargs)
    
    @staticmethod
    def evaluate(x, x_ref, x0, alpha, phi0):
        """Evaluate the model (static function)"""
        return phi0*np.power((x-x_ref)/x0, -alpha)


    def integral(self, range_min, range_max, scale="linear", **kwargs):
        """Integrates to 1 if normalise=True"""
        if scale=="log":
            integral = integrate_spectrum(self, range_min, range_max, **kwargs)
        else:
            integral = integrate_linear(self, range_min, range_max, **kwargs)
        return integral
        

### Compare spectral

In [3]:
# If correct units are given
p = PowerLawModel(input_units="TeV", norm_units='TeV-1 s-1 cm-2')
p.x_ref.value = 0.0
p.phi0.value = 1e-12
p.parameters.to_table()


type,name,value,unit,error,min,max,frozen,is_norm,link,prior
str1,str5,float64,str14,float64,float64,float64,bool,bool,str1,str1
,alpha,2.0000e+00,,0.000e+00,nan,nan,True,False,,
,x0,1.0000e+00,TeV,0.000e+00,nan,nan,True,False,,
,x_ref,0.0000e+00,TeV,0.000e+00,nan,nan,True,False,,
,phi0,1.0000e-12,TeV-1 s-1 cm-2,0.000e+00,nan,nan,True,True,,


In [4]:
energies = [1, 5, 10]*u.GeV
p(energies)

<Quantity [1.e-06, 4.e-08, 1.e-08] 1 / (TeV s cm2)>

In [5]:
#Compare with standard value
p1 = PowerLawSpectralModel()
p1(energies)

<Quantity [1.e-06, 4.e-08, 1.e-08] 1 / (TeV s cm2)>

In [6]:
p.integral(0.1 * u.TeV, 10 * u.TeV, scale="log")

<Quantity 9.9e-12 1 / (s cm2)>

In [7]:
p1.integral(0.1 * u.TeV, 10 * u.TeV)

<Quantity 9.9e-12 1 / (s cm2)>

### Compare temporal

In [8]:
# Normal temporal model
t_ref = Time(46300, format="mjd")
t1 = PowerLawTemporalModel(t_ref=t_ref.mjd * u.d, alpha=-2)
times = Time([46302, 46303], format="mjd")
time_start = t_ref + [1, 3, 5] * u.day
time_stop = t_ref + [2, 3.5, 6] * u.day
t1(time_start)


<Quantity [1.        , 0.11111111, 0.04      ]>

In [9]:
t = PowerLawModel(input_units=u.d)
t.x_ref.value = t_ref.mjd
t(time_start)

<Quantity [1.        , 0.11111111, 0.04      ]>

In [10]:
t1.integral(time_start, time_stop)

<Quantity [0.2       , 0.01904762, 0.01333333]>

In [11]:
t.integral(time_start, time_stop)

<Quantity [0.2025312 , 0.01924233, 0.01347028]>

In [12]:
## General axis
ph = PowerLawModel()
phases = [0.1, 0.3, 0.5]*u.dimensionless_unscaled
ph(phases)

<Quantity [1.2345679 , 2.04081633, 4.        ]>

In [13]:
# Fails as expected if units are mismatched
px = PowerLawModel()
px([3, 8]*u.TeV)

UnitConversionError: 'TeV' (energy/torque/work) and '' (dimensionless) are not convertible

## Using with a SkyModel

In [14]:
class SkyModelGeneral():

    def __init__(self, models, name="abc"):
        """Models should be a dict of underlying models"""
        self.models = models
        self.name = name # replace by make_name()

    def parameters(self):
        parameters = []
        for model in self.models:
            parameters.append(model.parameters)
        return Parameters.from_stack(parameters)

    def evaluate_geom(self, geom, gti=None):
        coord = geom.get_coord()
        eval = Map.from_geom(geom, data=1)
        for axis, model in self.models.items():
            ax = geom.axes[axis]
            if isinstance(ax, TimeMapAxis):
                val = self._compute_time_integral(geom, ax, model, gti)
            else:
                val = model(coord[axis])
            eval = eval*val
        return eval.quantity

    def _compute_time_integral(self, geom, time_axis, model, gti):
        """similar to SkyModel._compute_time_integral"""
        temp_eval = np.ones(time_axis.nbin)
        for idx in range(time_axis.nbin):
            #Doing only for the case of GTI = None
            t1, t2 = time_axis.time_min[idx], time_axis.time_max[idx]
            integ = model.integral(t1, t2)
            temp_eval[idx] = np.sum(integ)
        value = Map.from_geom(geom, data=1).data
        value = (value.T * temp_eval).T
        return value
        

    def integrate_geom(self, geom, gti=None, oversampling_factor=None):
        eval = Map.from_geom(geom, data=1)
        for axis, model in self.models.items():
            ax = geom.axes[axis]
            if isinstance(ax, TimeMapAxis):
                integ = self._compute_time_integral(geom, ax, model, gti)
            else:
                edges = ax.edges
                shape = len(geom.data_shape) * [1,]
                shape[geom.axes.index_data(axis)] = -1
                integ = model.integral(edges[:-1], edges[1:], scale=ax.interp).reshape(shape)
            eval = eval*integ
        return eval

In [15]:
energy_axis = MapAxis.from_energy_bounds(
        "1 TeV", "10 TeV", nbin=3, name="energy_true"
    )

time_min = t_ref + [1, 3, 5, 7] * u.day
time_max = t_ref + [2, 4, 6, 8] * u.day

time_axis = TimeMapAxis.from_time_edges(time_min=time_min, time_max=time_max)


#### Just spectral

In [16]:
reg_geom = RegionGeom.create(region=None,
        axes=[energy_axis],
    )

sky_model = SkyModel(
        spectral_model=p1
    )
sky_model_general = SkyModelGeneral(models={"energy_true": p})

In [17]:
sky_model.evaluate_geom(reg_geom)

<Quantity [[[4.64158883e-13]],

           [[1.00000000e-13]],

           [[2.15443469e-14]]] 1 / (TeV s cm2)>

In [18]:

sky_model_general.evaluate_geom(reg_geom)

<Quantity [[[4.64158883e-13]],

           [[1.00000000e-13]],

           [[2.15443469e-14]]] 1 / (TeV s cm2)>

In [19]:
sky_model.integrate_geom(reg_geom).quantity

<Quantity [[[5.35841117e-13]],

           [[2.48715414e-13]],

           [[1.15443469e-13]]] 1 / (s cm2)>

In [20]:
sky_model_general.integrate_geom(reg_geom).quantity

<Quantity [[[5.35841117e-13]],

           [[2.48715414e-13]],

           [[1.15443469e-13]]] 1 / (s cm2)>

#### Temporal & Spectral

In [21]:
reg_geom = RegionGeom.create(region=None,
        axes=[energy_axis,time_axis],
    )

In [22]:
sky_model = SkyModel(
        spectral_model=p1,
        temporal_model=t1
    )
sky_model_general = SkyModelGeneral(models={"energy_true": p, "time":t})

In [23]:
sky_model.evaluate_geom(reg_geom)

<Quantity [[[[2.32079442e-13]],

            [[5.00000000e-14]],

            [[1.07721735e-14]]],


           [[[3.86799069e-14]],

            [[8.33333333e-15]],

            [[1.79536224e-15]]],


           [[[1.54719628e-14]],

            [[3.33333333e-15]],

            [[7.18144897e-16]]],


           [[[8.28855149e-15]],

            [[1.78571429e-15]],

            [[3.84720480e-16]]]] 1 / (TeV s cm2)>

In [24]:
sky_model_general.evaluate_geom(reg_geom)

<Quantity [[[[2.35016644e-13]],

            [[5.06328010e-14]],

            [[1.09085063e-14]]],


           [[[3.90870615e-14]],

            [[8.42105212e-15]],

            [[1.81426068e-15]]],


           [[[1.56308765e-14]],

            [[3.36757027e-15]],

            [[7.25521020e-16]]],


           [[[8.37302935e-15]],

            [[1.80391449e-15]],

            [[3.88641595e-16]]]] 1 / (TeV s cm2)>

In [25]:
sky_model.integrate_geom(reg_geom).data

array([[[[2.67920558e-13]],

        [[1.24357707e-13]],

        [[5.77217345e-14]]],


       [[[4.46534264e-14]],

        [[2.07262845e-14]],

        [[9.62028908e-15]]],


       [[[1.78613706e-14]],

        [[8.29051381e-15]],

        [[3.84811563e-15]]],


       [[[9.56859137e-15]],

        [[4.44134668e-15]],

        [[2.06149052e-15]]]])

In [26]:
sky_model_general.integrate_geom(reg_geom).data

array([[[[2.71311366e-13]],

        [[1.25931581e-13]],

        [[5.84522620e-14]]],


       [[[4.51234597e-14]],

        [[2.09444547e-14]],

        [[9.72155470e-15]]],


       [[[1.80448261e-14]],

        [[8.37566634e-15]],

        [[3.88763994e-15]]],


       [[[9.66611554e-15]],

        [[4.48661340e-15]],

        [[2.08250146e-15]]]])

#### Can work without a spectral model

In [27]:
phase_axis = MapAxis.from_edges([0.1, 0.2, 0.5], name="phase")
reg_geom = RegionGeom.create(region=None,
        axes=[phase_axis])

In [28]:
ph = PowerLawModel()
sky_model_general = SkyModelGeneral(models={"phase": ph})

In [29]:
sky_model_general.evaluate_geom(reg_geom)

<Quantity [[[1.38408304]],

           [[2.36686391]]]>

In [30]:
sky_model_general.integrate_geom(reg_geom).data

array([[[0.35075412]],

       [[1.89609321]]])